In [1]:
import sys
import pyspark.sql
from pyspark.sql import functions as F
from pyspark.sql.types import BooleanType

24/06/01 21:55:07 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [2]:
spark = SparkSession\
        .builder\
        .appName("Modulo 3 Desafio")\
        .enableHiveSupport()\
        .getOrCreate()

24/06/01 21:56:31 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [3]:
cnaes = '/home/telma/xpe/engenharia_dados/dados/cnaes/cnaes.csv'
estab1 = '/home/telma/xpe/engenharia_dados/dados/estabelecimentos/estabelecimentos/estabelecimentos-1.csv'
estab2 = '/home/telma/xpe/engenharia_dados/dados/estabelecimentos/estabelecimentos/estabelecimentos-2.csv'
estab3 = '/home/telma/xpe/engenharia_dados/dados/estabelecimentos/estabelecimentos/estabelecimentos-3.csv'

In [4]:
df_cnae = spark.read.csv(cnaes, header=True, inferSchema=True, sep=';')

In [5]:
df_estab1 = spark.read.csv(estab1, header=True, inferSchema=True, sep=';')

In [6]:
df_estab2 = spark.read.csv(estab2, header=True, inferSchema=True, sep=';')

In [7]:
df_estab3 = spark.read.csv(estab3, header=True, inferSchema=True, sep=';')

In [8]:
df_estab = df_estab1.union(df_estab2.union(df_estab3))

In [9]:
df_estab.count()

20996744

In [10]:
df_estab.createOrReplaceTempView("estab")

24/06/01 21:58:18 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


### 1) Quantos estabelecimentos existem?

In [11]:
spark.sql("""select count(distinct CNPJ_BASICO,CNPJ_ORDEM,CNPJ_DV) from estab """).show()

24/06/01 22:00:03 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/06/01 22:00:03 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/06/01 22:00:06 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/06/01 22:00:06 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/06/01 22:00:14 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/06/01 22:00:14 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/06/01 22:00:16 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


+------------------------------------------------+
|count(DISTINCT CNPJ_BASICO, CNPJ_ORDEM, CNPJ_DV)|
+------------------------------------------------+
|                                        20996744|
+------------------------------------------------+



### 2) Quantas colunas existem na tabela estabelecimentos e quantas são identificadas como números?

In [14]:
# QTDE TOTAL DE COLUNAS
len(df_estab.columns)

30

In [24]:
# Quantidade de colunas identificadas como númericas.
x = 0

for i in df_estab.columns:
    if dict(df_estab.dtypes)[i] != 'string':
        x = x + 1

print(x)

13


### 3) Gere arquivo parquet a partir de "estabelecimentos" e compare o tamanho dos arquivos parquet e csv originais.

In [ ]:
df_estab.write\
        .parquet('/home/telma/xpe/engenharia_dados/dados/estabelecimentos/estabelecimentos/estabelecimentos.parquet',
                mode = 'overwrite')

# .csv     -->> 3,781Gb
# .parquet -->> 1,5Gb
# (3,781 - 1,5)/1,5 = 

In [25]:
# Economia de espaço

csv = 3.781
parquet = 1.5

economia = (csv - parquet)/parquet

economia

1.5206666666666668

### 4) Spark.sql ==>> Quantos estabelecimentos não tem logradouro informado.

In [26]:
spark.sql("select count(*) from estab where logradouro is null").show()

+--------+
|count(1)|
+--------+
|     828|
+--------+



### 5) Quantos estabelecimentos ficam localizados numa avenida?

In [27]:
def is_avenida(logradouro):
    if str(logradouro).upper().startswith('AVENIDA'):
        return True
    else:
        return False

In [28]:
# is_avenidaUDF = F.udf(lambda logradouro: is_avenida(logradouro), BooleanType())
spark.udf.register("is_avenidaUDF", is_avenida, BooleanType())

<function __main__.is_avenida(logradouro)>

In [29]:
spark.sql("""
          select count(*) from estab where is_avenidaUDF(logradouro) = True
          """).show()

+--------+
|count(1)|
+--------+
|   52587|
+--------+



### 6) Quantos CEP's distintos existem em estabelecimentos?

In [ ]:
###  Resposta 958396 não existe

# spark.sql("select count(distinct CEP) from estab").show()
df_estab.select('CNPJ_BASICO', 'CNPJ_ORDEM', 'CNPJ_DV', 'CEP')\
        .agg(F.countDistinct(df_estab.CEP).alias("QT_CEPs"))\
        .show()

### 7) Quantos CNAEs existem na tabela de CNAEs

In [30]:
df_cnae.createOrReplaceTempView("cnae")

In [31]:
spark.sql("select count(distinct cnae) from cnae").show()

+--------------------+
|count(DISTINCT cnae)|
+--------------------+
|                1359|
+--------------------+



### 8) Quantos estabelecimentos estão relacionados a cultivo?

In [32]:
def is_cnae_cultivo(desc_cnae):
    if str(desc_cnae).upper().startswith("CULTIVO"):
        return True
    else:
        return False

In [33]:
spark.udf.register("is_cnae_cultivoUDF", is_cnae_cultivo, BooleanType())

<function __main__.is_cnae_cultivo(desc_cnae)>

In [34]:
spark.sql("""
select count(*) from estab e inner join cnae c 
    on e.CNAE_PRINCIPAL = c.CNAE 
  --  OR e.CNAE_SECUNDARIA = c.CNAE
WHERE is_cnae_cultivoUDF(c.DESCRICAO_CNAE) = True
""").show()

+--------+
|count(1)|
+--------+
|  200243|
+--------+



### 11) Quantos estabelecimentos são filiais?

In [35]:
spark.sql("select count(distinct CNPJ_BASICO, CNPJ_ORDEM, CNPJ_DV) from estab where IDENTIFICADOR_MATRIZ_FILIAL =  2").show()

+------------------------------------------------+
|count(DISTINCT CNPJ_BASICO, CNPJ_ORDEM, CNPJ_DV)|
+------------------------------------------------+
|                                         1093082|
+------------------------------------------------+

